In [1]:
!pip install yfinance
!pip install yahooquery
!pip install tvdatafeed
!pip install tensorflow
!pip install --upgrade mplfinance
from tvDatafeed import TvDatafeed, Interval
from yahooquery import Screener
import yfinance as yf   
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
import random 
from tensorflow.keras.models import load_model
import os
import shutil
import mplfinance as mpl 
from datetime import datetime
import glob
from PIL import Image
import cv2
import gc
import gc
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib
matplotlib.use('agg')
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,Activation,Dropout,Flatten,Dense,AveragePooling2D,GlobalAveragePooling2D
!pip install -U git+https://github.com/twopirllc/pandas-ta
import pandas_ta as pta


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 6.4 MB 18.9 MB/s 
     |████████████████████████████████| 62 kB 1.7 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.28.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |██████████

In [2]:

def work_with_dir():
  if os.path.exists("/content/data/"):
    shutil.rmtree("/content/data/", ignore_errors=True)
    print("Data Folder Removed")
    os.mkdir("/content/data/")
  if not os.path.exists("/content/data/"):
    os.mkdir("/content/data/")
  if not os.path.exists("/content/extracted/"):
    os.mkdir("/content/extracted/")
  if not os.path.exists("/content/checkpoints/"):
    os.mkdir("/content/checkpoints/")


def get_crypto_syms():
   # 'all_cryptocurrencies_au','all_cryptocurrencies_ca','all_cryptocurrencies_eu','all_cryptocurrencies_gb','all_cryptocurrencies_in',
   screens = [
       'all_cryptocurrencies_us', 'all_cryptocurrencies_au', 'all_cryptocurrencies_ca', 'all_cryptocurrencies_eu', 'all_cryptocurrencies_gb', 'all_cryptocurrencies_in', ]
   s = Screener()
   symbols = []
   for i in screens:
      data = s.get_screeners(i, count=250)
      dicts = data[i]['quotes']
      syms = [d['symbol'] for d in dicts]
      for sym in syms:
        symbols.append(sym)
   # print(len(symbols))
   # pieces = 15
   # new_arrays = np.array_split(symbols, pieces)
   return symbols


def download_data(symbols, periodd, intervall):
  indexx = 100
  work_with_dir()
  for symbol in symbols:
    if ((symbols.index(symbol)+1) % 100 == 0):
      print(f" -- {indexx}", end="")
      indexx = indexx + 100
    try:
        data = yf.download(symbol, period=periodd,
                           interval=intervall, progress=False, show_errors=False)
        if data.empty:
           pass
        else:
            data.to_csv(f"/content/data/{symbol}.csv")
    except:
       print("Error!")
  print(" ")


def extract_data(how_many_future_candles, how_many_past_candles, each_row_past):
  print(f"Files In Data : {len(os.listdir('/content/data/'))}")
  pd.options.mode.chained_assignment = None
  files = os.listdir("/content/data/")
  print("Processing File:")
  now = datetime.now().strftime("%H%M%S")
  os.mkdir(f"/content/extracted/{now}/")
  index = 1
  for file in files:
     print(f"File Number {index}:", end=" ")
     each_file_proc(file, now, how_many_future_candles,
                    how_many_past_candles, each_row_past)
     index = index + 1
  print(" ")
  return now


def each_file_proc(file, now, how_many_future_candles, how_many_past_candles, each_row_past):

    address = f"/content/data/{file}"
    data = pd.read_csv(address)
    data = data.dropna()
    dfs = []
    suggs = []
    if len(data.columns) == 7:
      data = data.iloc[:, 1:]
    data["6_rsi"] =  pta.rsi(data['Close'], length = 6)
    data["14_rsi"] = pta.rsi(data['Close'], length = 14)
    data["26_rsi"] = pta.rsi(data['Close'], length = 26)
    data["52_rsi"] = pta.rsi(data['Close'], length = 52)

    # drop Adj Close and Volume
    data.drop(columns=['Adj Close', 'Volume'], inplace=True)
    data = np.array(data)
    data = data.astype(float)
    for i in range(each_row_past, data.shape[0]-which_future_or_past):

        rows = data[i-each_row_past:i, :]
        past_candles = []
        for z in range(1, how_many_past_candles+1):
          past_candles.append((data[i-z][3]+data[i-z][0])/2)
        past_candles = sum(past_candles)/len(past_candles)
        next_candles = []
        for z in range(0, how_many_future_candles):
          next_candles.append((data[i+z][3]+data[i+z][0])/2)
        next_candles = sum(next_candles)/len(next_candles)
        if next_candles > past_candles:
          suggs.append(1)
        else:
          suggs.append(0)

        x = pd.DataFrame(rows, columns=["Open", "High", "Low", "Close","6_rsi", "14_rsi", "26_rsi","52_rsi"])
        del rows
        del next_candles
        dfs.append(x)
        del x
    del data

    index = 0
    all_n = np.array(dfs).shape[0]
    for df in dfs:
        date_list = []
        rrng = np.array(df).shape[0]
        for i in range(rrng):
            date_list.append(datetime.fromordinal(
                datetime.today().toordinal()+i).strftime('%Y-%m-%d'))
        df.index = date_list
        df.index.name = "Date"
        df.index = pd.to_datetime(df.index)
        right_now = datetime.now().strftime("%H%M%S%f")
        address = f"/content/extracted/{now}/{right_now}_{suggs[index]}.png"
        
        #apds = [ 
        # mpl.make_addplot(df["6_rsi"],color='b',width=1),
        # mpl.make_addplot(df["14_rsi"],color='g',width=1),
        # mpl.make_addplot(df["26_rsi"],color='r',width=1),
        # mpl.make_addplot(df["52_rsi"],color='y',width=1),
       #]


        mpl.plot(df, type="candle", style="yahoo", axisoff=True,
                            savefig=address, tight_layout=True,figsize =(4,3))
        
        plt.close("all")
        del df
        del date_list
        del rrng
        del i
        del right_now
        gc.collect()



        index = index + 1
        if index % 10 == 0:
            print(f"{index}/{all_n}", end=" ")
        if index % 160 == 0:
          print(" ")

    print("")
    del dfs
    del suggs
    del index
    del all_n
    gc.collect()



def start(how_many_future_candles, how_many_past_candles, each_row_past):
    folder_name = extract_data(
        how_many_future_candles, how_many_past_candles, each_row_past)
    return folder_name


In [3]:
symbols = get_crypto_syms()
print(f"Symbols : {len(symbols)}")
#symbols = ["btc-usd","eth-usd","trx-usd","ltc-usd","xrp-usd","bnb-usd","FTT-USD"]
symbols = ["btc-usd"]
download_data(symbols,"100d","1h")

Symbols : 1500
 


In [4]:
how_many_future_candles = 5
how_many_past_candles = 1
each_row_past = 90

global which_future_or_past
which_future_or_past = None
if how_many_future_candles > how_many_past_candles:
    which_future_or_past = how_many_future_candles
else:
    which_future_or_past = how_many_past_candles

folder_name = start(how_many_future_candles,how_many_past_candles,each_row_past)
len(os.listdir(f"/content/extracted/{folder_name}"))

Files In Data : 1
Processing File:
File Number 1: 10/2291 20/2291 30/2291 40/2291 50/2291 60/2291 70/2291 80/2291 90/2291 100/2291 110/2291 120/2291 130/2291 140/2291 150/2291 160/2291  
170/2291 180/2291 190/2291 200/2291 210/2291 220/2291 230/2291 240/2291 250/2291 260/2291 270/2291 280/2291 290/2291 300/2291 310/2291 320/2291  
330/2291 340/2291 350/2291 360/2291 370/2291 380/2291 390/2291 400/2291 410/2291 420/2291 430/2291 440/2291 450/2291 460/2291 470/2291 480/2291  
490/2291 500/2291 510/2291 520/2291 530/2291 540/2291 550/2291 560/2291 570/2291 580/2291 590/2291 600/2291 610/2291 620/2291 630/2291 640/2291  
650/2291 660/2291 670/2291 680/2291 690/2291 700/2291 710/2291 720/2291 730/2291 740/2291 750/2291 760/2291 770/2291 780/2291 790/2291 800/2291  
810/2291 820/2291 830/2291 840/2291 850/2291 860/2291 870/2291 880/2291 890/2291 900/2291 910/2291 920/2291 930/2291 940/2291 950/2291 960/2291  
970/2291 980/2291 990/2291 1000/2291 1010/2291 1020/2291 1030/2291 1040/2291 1050/2

2291

In [5]:
#folder_name = '122227'



label = []
data  = []
files = os.listdir(f"/content/extracted/{folder_name}/")
for i, image_name in enumerate(files):
  if image_name.split(".")[1] == "png":
    image = cv2.imread(f"/content/extracted/{folder_name}"+"/"+image_name)
    dim = (256, 128)
    resized = cv2.resize(image, dim)
    data.append(np.array(resized))
    sugg = image_name.split("_")[1].split(".")[0]
    label.append(int(sugg))
data = np.array(data)
data = data / 255
print(data.shape)
xTrain , xTest , yTrain , yTest = train_test_split(data,label,test_size=0.2,random_state=99)

data = None
label = None
print(f"xTrain : {len(xTrain)} \\ xTest : {len(xTest)}")

nytrain = []
nytest = []
yn = 0
nn = 0

for i in yTrain:
  if i == 1:
    nytrain.append([1,0])
    yn += 1
  else:
    nytrain.append([0,1])
    nn += 1
for i in yTest:
  if i == 1:
    nytest.append([1,0])
    yn += 1
  else:
    nytest.append([0,1])
    nn += 1

yTrain = np.array(nytrain)
yTest = np.array(nytest)

print(f"yn: {yn} nn: {nn}")

(2291, 128, 256, 3)
xTrain : 1832 \ xTest : 459
yn: 1109 nn: 1182


In [9]:
model = Sequential()

model.add(Conv2D(64,    (4, 4),activation="relu", input_shape=(xTrain.shape[1], xTrain.shape[2],3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(100,   (3, 3),activation="relu",))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(200,   (3, 3),activation="relu",))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128,   (2, 2),activation="relu",))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(1024,activation="relu"))
model.add(Dense(1024,activation="relu"))
model.add(Dense(1024,activation="relu"))
model.add(Dense(1024,activation="relu"))
model.add(Dense(2,activation="sigmoid"))

adamax = tf.keras.optimizers.Adamax(
    learning_rate=0.001)
model.compile(loss='categorical_crossentropy',
              optimizer=adamax,
              metrics=['accuracy'])

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_7 (Conv2D)           (None, 125, 253, 64)      3136      
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 62, 126, 64)      0         
 2D)                                                             
                                                                 
 conv2d_8 (Conv2D)           (None, 60, 124, 100)      57700     
                                                                 
 max_pooling2d_8 (MaxPooling  (None, 30, 62, 100)      0         
 2D)                                                             
                                                                 
 conv2d_9 (Conv2D)           (None, 28, 60, 200)       180200    
                                                                 
 max_pooling2d_9 (MaxPooling  (None, 14, 30, 200)     

In [11]:
filepath = "/content/checkpoints/{epoch:02d}-{val_accuracy:.2f}.h5"
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=filepath,
    save_weights_only=False,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

#model.fit(xTrain,yTrain,batch_size=64,epochs=30,validation_data=(xTest,yTest), callbacks=model_checkpoint_callback)
model.fit(xTrain,yTrain,batch_size=32,epochs=20,validation_data=(xTest,yTest))


Epoch 1/20
58/58 [==============================] - 198s 3s/step - loss: 0.0134 - accuracy: 0.9951 - val_loss: 1.0681 - val_accuracy: 0.7865
Epoch 2/20
58/58 [==============================] - 239s 4s/step - loss: 9.0653e-04 - accuracy: 1.0000 - val_loss: 1.2205 - val_accuracy: 0.7887
Epoch 3/20
58/58 [==============================] - 195s 3s/step - loss: 2.0945e-04 - accuracy: 1.0000 - val_loss: 1.3894 - val_accuracy: 0.7887
Epoch 4/20
58/58 [==============================] - 193s 3s/step - loss: 8.2700e-05 - accuracy: 1.0000 - val_loss: 1.5069 - val_accuracy: 0.7843
Epoch 5/20
58/58 [==============================] - 193s 3s/step - loss: 4.2710e-05 - accuracy: 1.0000 - val_loss: 1.6031 - val_accuracy: 0.7843
Epoch 6/20
58/58 [==============================] - 195s 3s/step - loss: 2.4283e-05 - accuracy: 1.0000 - val_loss: 1.6795 - val_accuracy: 0.7843
Epoch 7/20
58/58 [==============================] - 198s 3s/step - loss: 1.5190e-05 - accuracy: 1.0000 - val_loss: 1.7438 - val_accura